**Dataset**
labeled datasset collected from Spotify (Assignment 1 - Spotify Reviews Rating)

**Objective**
classify Review to a category from 1 to 5. <br>

**Total Estimated Time = 90-120 Mins**

**Evaluation metric**
macro f1 score

### Import used libraries

In [1]:
pip install contractions

In [2]:
pip install -U pip setuptools wheel


In [3]:
pip install -U spacy

In [4]:
! python -m spacy download en_core_web_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 27.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
import re
import contractions
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from tqdm.auto import tqdm
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import spacy
import nltk


In [6]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load Dataset

In [7]:
data= pd.read_csv ('Assignment 1 - Spotify Reviews Rating.csv')



  ✉ Preview data

In [8]:
data.head(10)

,Time_submitted,Review,Rating
0,7/9/2022 15:00,"Great music service, the audio is high quality...",5
1,7/9/2022 14:21,Please ignore previous negative rating. This a...,5
2,7/9/2022 13:27,"This pop-up ""Get the best Spotify experience o...",4
3,7/9/2022 13:26,Really buggy and terrible to use as of recently,1
4,7/9/2022 13:20,Dear Spotify why do I get songs that I didn't ...,1
5,7/9/2022 13:20,The player controls sometimes disappear for no...,3
6,7/9/2022 13:19,I love the selection and the lyrics are provid...,5
7,7/9/2022 13:17,Still extremely slow when changing storage to ...,3
8,7/9/2022 13:16,It's a great app and the best mp3 music app I ...,5
9,7/9/2022 13:11,"I'm deleting this app, for the following reaso...",1


### Data splitting

It is a good practice to split the data before EDA helps maintain the integrity of the machine learning process, prevents data leakage, simulates real-world scenarios more accurately, and ensures reliable model performance evaluation on unseen data.

In [9]:
#Extracting x, y from dataframe
x = data['Review']
y = data['Rating']

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify = y, random_state = 42)

### EDA on training data

In [11]:
# Concat x train and y train
train_data = pd.DataFrame(pd.concat([x_train, y_train], axis=1))


In [12]:
train_data


,Review,Rating
9780,Making it different from other's is Podcast......,5
51477,Great app! A loads if music and fun! Thank you!,5
550,Best music app so far. I am a premium customer.,5
42155,It is so good because any sing you say they ha...,5
17895,I love spotify. It says it's downloaded.. Why ...,2
...,...,...
34066,Been using it for three years and the sessions...,1
12296,This app sucks. You can only choose what you w...,1
13187,I was YT music user before and I left that app...,5
37787,For some time now the app keeps acting wierd. ...,2


- check NaNs

In [13]:
pd.isnull(train_data).sum()

Review    0
Rating    0
dtype: int64

In [14]:
#No Null Values

- check duplicates

In [15]:
train_data.duplicated().sum()

143

show a representative sample of data texts to find out required preprocessing steps

In [16]:
sample = train_data.sample(min(15, len(train_data)))
print("Sample Text Data:")
print(sample)

Sample Text Data:
                                                  Review  Rating
30544  I literally can't pause. The only way to stop ...       1
43315  Great customer service and they had a lot of p...       4
58137  Thank you Spotify for holding strong and not c...       5
50820  Spotify is good app but it not playing song wh...       1
11267       Good over all. Nice selection to choose from       5
13580  M ñot able to login pls help , they say, pls c...       4
18066  It's gotta be the most convenient and operate ...       5
57797  Nothing but positive experiences with this app...       5
12093                         Best music and podcast app       5
6900   Fantastic! Love making my own playlists and be...       5
18902           Awesome but can reduce the number of ads       5
58816  Can't really find the song/music you are looki...       1
6096   This stupid app keeps turning itself off in th...       2
32083  I'll give y'all a 3 not because I hate it. Thi...       3
1961   

- check dataset balancing

In [17]:
for i in range(1,6):
  print(f"class{i}", (train_data['Rating'] == i).sum()/len(train_data))

class1 0.28659563673262306
class2 0.11555555555555555
class3 0.11180111618467783
class4 0.12732623033992896
class5 0.3587214611872146



 ♻  Cleaning and Preprocessing Steps are:  
 ---
  1.   Drop Duplicates
  2.   Remove Emojis
  3.   Text Normalization
  4.   Remove newlines and tabs
  5.   Remove Extra Spaces
  6.   Remove contractions
  7.   Remove Punctuation
  8.   Remove Stopwords



### Cleaning and Preprocessing

---
⚓ REMOVE DUPLICTES

In [18]:
train_data = train_data.drop_duplicates()
train_data

,Review,Rating
9780,Making it different from other's is Podcast......,5
51477,Great app! A loads if music and fun! Thank you!,5
550,Best music app so far. I am a premium customer.,5
42155,It is so good because any sing you say they ha...,5
17895,I love spotify. It says it's downloaded.. Why ...,2
...,...,...
34066,Been using it for three years and the sessions...,1
12296,This app sucks. You can only choose what you w...,1
13187,I was YT music user before and I left that app...,5
37787,For some time now the app keeps acting wierd. ...,2


---
⚓ REMOVE Emojis

In [19]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001F004-\U0001F9CF"  # Miscellaneous Symbols and Pictographs
                           u"\U0001F600-\U0001F64F"  # Emoticons (iOS)
                           u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols (iOS)
                           u"\U0001F300-\U0001F5FF"  # Other additional symbols (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


In [20]:
review_temporary = train_data['Review'].copy()

In [21]:
review_temporary= review_temporary.astype(str).apply(remove_emoji)
review_temporary

9780     Making it different from other's is Podcast......
51477      Great app! A loads if music and fun! Thank you!
550        Best music app so far. I am a premium customer.
42155    It is so good because any sing you say they ha...
17895    I love spotify. It says it's downloaded.. Why ...
                               ...                        
34066    Been using it for three years and the sessions...
12296    This app sucks. You can only choose what you w...
13187    I was YT music user before and I left that app...
37787    For some time now the app keeps acting wierd. ...
45329    Actually I wish to not give a single star to s...
Name: Review, Length: 49132, dtype: object

In [22]:
sample = review_temporary.sample(min(15, len(review_temporary)))
print("Sample After remove emoji:")
print(sample)

Sample After remove emoji:
3462     I get to hear two songs, and then get 3 ads in...
40164    Love it! I get to listen to my fav songs but y...
22794    Need more recommendations on contemporary hits...
44321    Surprisingly the adds are not very intuitive. ...
10526    Without this app I don't think I would listen ...
40070    Been using it since 2015 if I can last recall ...
7060     Its a great app! Has worked perfectly across m...
42391    I love Spotify because you are able to make yo...
13974    Spotify has become super glitchy. I'm a paid m...
42976    if you don't have an account on you will only ...
36453    My Spotify is randomly pausing my music for a ...
10997    Plays awesome music. Just what the doctor orde...
20736    Decent price , could use more audio books thou...
6098     I hate this app now like seriously after the n...
38686    Won't play music isi get a text, it doesn't re...
Name: Review, dtype: object


---
⚓ Text Normalization

In [23]:
#Lower casing
def lower_case(text):
  text_lowercase = text.str.lower()
  return text_lowercase

In [24]:
review_temporary = lower_case(review_temporary)
review_temporary

9780     making it different from other's is podcast......
51477      great app! a loads if music and fun! thank you!
550        best music app so far. i am a premium customer.
42155    it is so good because any sing you say they ha...
17895    i love spotify. it says it's downloaded.. why ...
                               ...                        
34066    been using it for three years and the sessions...
12296    this app sucks. you can only choose what you w...
13187    i was yt music user before and i left that app...
37787    for some time now the app keeps acting wierd. ...
45329    actually i wish to not give a single star to s...
Name: Review, Length: 49132, dtype: object

Note >> stemming is the next process of Normalization, it will be done after removing contractions

---
⚓ Remove Lines & Tabs & Extra Spaces

In [25]:
def remove_lines_tabs_extraspaces(text):
    # Replace newlines and tabs with a single space
    text = re.sub(r'[\t\n\r]+', ' ', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [26]:
# Apply remove_lines_tabs_extraspaces
review_temporary= review_temporary.astype(str).apply(remove_lines_tabs_extraspaces)
review_temporary

9780     making it different from other's is podcast......
51477      great app! a loads if music and fun! thank you!
550        best music app so far. i am a premium customer.
42155    it is so good because any sing you say they ha...
17895    i love spotify. it says it's downloaded.. why ...
                               ...                        
34066    been using it for three years and the sessions...
12296    this app sucks. you can only choose what you w...
13187    i was yt music user before and i left that app...
37787    for some time now the app keeps acting wierd. ...
45329    actually i wish to not give a single star to s...
Name: Review, Length: 49132, dtype: object

---
⚓ Remove contractions

In [27]:
def remove_contractions(text):
    preprocessed_text = contractions.fix(text)
    return preprocessed_text


In [28]:
# Apply remove_contractions
review_temporary= review_temporary.apply(remove_contractions)
review_temporary

9780     making it different from other's is podcast......
51477      great app! a loads if music and fun! thank you!
550        best music app so far. i am a premium customer.
42155    it is so good because any sing you say they ha...
17895    i love spotify. it says it is downloaded.. why...
                               ...                        
34066    been using it for three years and the sessions...
12296    this app sucks. you can only choose what you w...
13187    i was yt music user before and i left that app...
37787    for some time now the app keeps acting wierd. ...
45329    actually i wish to not give a single star to s...
Name: Review, Length: 49132, dtype: object

---
⚓ Remove Punctuation

In [29]:
def remove_punctuation(text):
  output_text = text.translate(str.maketrans('', '', string.punctuation))
  return output_text

In [30]:
# Apply remove_punctuation
review_temporary= review_temporary.apply(remove_punctuation)
review_temporary

9780     making it different from others is podcast lov...
51477         great app a loads if music and fun thank you
550          best music app so far i am a premium customer
42155    it is so good because any sing you say they ha...
17895    i love spotify it says it is downloaded why ca...
                               ...                        
34066    been using it for three years and the sessions...
12296    this app sucks you can only choose what you wa...
13187    i was yt music user before and i left that app...
37787    for some time now the app keeps acting wierd i...
45329    actually i wish to not give a single star to s...
Name: Review, Length: 49132, dtype: object

In [31]:
sample = review_temporary.sample(min(15, len(review_temporary)))
print("Sample :")
print(sample)

Sample :
13426    horrible it adds songs thay i do not want in m...
21620           genuinely very useful and entertaining app
54119    why is not working in my phone it is showing t...
12241    easy to install ready to listen to some good m...
15629    love it plus i find music i have not heard in ...
36474    app crashes no play bar unable to play music s...
41582    very great for listening at low internet and b...
45101    i realy like it because you get to listen to s...
51587    spotify has a lot of great music to listen to ...
43965    the ads are nuts i was fine with having to hea...
1403                                   great app for music
47120    asking for review i was listening to music in ...
25878    please contact me and tell me why the shuffle ...
40887    i really like it i can pick my favorite and li...
48247    bro they need to make the playlist back to nor...
Name: Review, dtype: object


---
⚓ Remove Stop words

In [32]:
stop_words = set(stopwords.words('english'))

def remove_stop_words(text):
    words = word_tokenize(text)
    # Remove stop words
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Reconstruct the text without stop words
    return ' '.join(filtered_words)


In [33]:
review_temporary= review_temporary.apply(remove_stop_words)

In [34]:
review_temporary

9780              making different others podcast love app
51477                      great app loads music fun thank
550                    best music app far premium customer
42155    good sing say lyrics abd everything plus hear ...
17895    love spotify says downloaded see says multiple...
                               ...                        
34066    using three years sessions system getting wors...
12296    app sucks choose want hear choose want hear an...
13187    yt music user left app pay play music backgrou...
37787    time app keeps acting wierd plays without able...
45329    actually wish give single star spotify thought...
Name: Review, Length: 49132, dtype: object

In [35]:
x_train = train_data['review'] = review_temporary

<ipython-input-35-4d38ea604549>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train = train_data['review'] = review_temporary


In [36]:
y_train = train_data['Rating']



#**⚓ Same preprocessing for test data**

In [37]:
# Concat x test and y test
test_data = pd.DataFrame(pd.concat([x_test, y_test], axis=1))


In [38]:
review_temporary_test = test_data['Review'].copy()

In [39]:
review_temporary_test= review_temporary_test.astype(str).apply(remove_emoji)
review_temporary_test

6393     When I signed up it said I was offline while I...
37871    the current playing song doesn't show after up...
8789     Works pretty well, good music selection. Has t...
31661    Usually I love spotify and I've had premium fo...
41043    I love this app but add's I am done by it.... ...
                               ...                        
26453    Just curious how I can make Spotify (during ra...
27702    I have been using Spotify very happily for sev...
47339    Every time I go into my library, it says "your...
28037    Keeps crashing! I absolutely love spotify, I'm...
45708    Good app but need some optimization. Overall n...
Name: Review, Length: 12319, dtype: object

In [40]:
review_temporary_test = lower_case(review_temporary_test)
review_temporary_test

6393     when i signed up it said i was offline while i...
37871    the current playing song doesn't show after up...
8789     works pretty well, good music selection. has t...
31661    usually i love spotify and i've had premium fo...
41043    i love this app but add's i am done by it.... ...
                               ...                        
26453    just curious how i can make spotify (during ra...
27702    i have been using spotify very happily for sev...
47339    every time i go into my library, it says "your...
28037    keeps crashing! i absolutely love spotify, i'm...
45708    good app but need some optimization. overall n...
Name: Review, Length: 12319, dtype: object

In [41]:
# Apply remove_lines_tabs_extraspaces
review_temporary_test= review_temporary_test.astype(str).apply(remove_lines_tabs_extraspaces)
review_temporary_test

6393     when i signed up it said i was offline while i...
37871    the current playing song doesn't show after up...
8789     works pretty well, good music selection. has t...
31661    usually i love spotify and i've had premium fo...
41043    i love this app but add's i am done by it.... ...
                               ...                        
26453    just curious how i can make spotify (during ra...
27702    i have been using spotify very happily for sev...
47339    every time i go into my library, it says "your...
28037    keeps crashing! i absolutely love spotify, i'm...
45708    good app but need some optimization. overall n...
Name: Review, Length: 12319, dtype: object

In [42]:
# Apply remove_contractions
review_temporary_test= review_temporary_test.apply(remove_contractions)
review_temporary_test

6393     when i signed up it said i was offline while i...
37871    the current playing song does not show after u...
8789     works pretty well, good music selection. has t...
31661    usually i love spotify and i have had premium ...
41043    i love this app but add's i am done by it.... ...
                               ...                        
26453    just curious how i can make spotify (during ra...
27702    i have been using spotify very happily for sev...
47339    every time i go into my library, it says "your...
28037    keeps crashing! i absolutely love spotify, i a...
45708    good app but need some optimization. overall n...
Name: Review, Length: 12319, dtype: object

In [43]:
# Apply remove_punctuation
review_temporary_test= review_temporary_test.apply(remove_punctuation)
review_temporary_test

6393     when i signed up it said i was offline while i...
37871    the current playing song does not show after u...
8789     works pretty well good music selection has the...
31661    usually i love spotify and i have had premium ...
41043    i love this app but adds i am done by it after...
                               ...                        
26453    just curious how i can make spotify during ran...
27702    i have been using spotify very happily for sev...
47339    every time i go into my library it says your l...
28037    keeps crashing i absolutely love spotify i am ...
45708    good app but need some optimization overall ni...
Name: Review, Length: 12319, dtype: object

In [44]:
review_temporary_test= review_temporary_test.apply(remove_stop_words)

In [45]:
x_test  = review_temporary_test

---
**⚓ Word Embedding Using Spacy**

In [46]:
nlp = spacy.load("en_core_web_md")


In [47]:
x_train_v = np.zeros((len(x_train), 300))
x_test_v = np.zeros((len(x_test), 300))

for i, doc in tqdm(enumerate(nlp.pipe(x_train)), total=len(x_train)):
    x_train_v[i, :] = doc.vector

for i, doc in tqdm(enumerate(nlp.pipe(x_test)), total=len(x_test)):
    x_test_v[i, :] = doc.vector

  0%|          | 0/49132 [00:00<?, ?it/s]

  0%|          | 0/12319 [00:00<?, ?it/s]

**You  are doing Great so far!**

### Modelling

In [48]:
categories =['1','2','3','4','5']

In [ ]:
clf = LinearSVC(class_weight = 'balanced' , max_iter = 4000)
clf.fit(x_train_v, y_train)
print(classification_report(y_test, clf.predict(x_test_v), target_names=categories))

In [56]:
class_weights = {1:10, 2:70, 3:70, 4:50,5:10 }

In [80]:
clf2 = LogisticRegression(class_weight = class_weights, max_iter = 3000)
clf2.fit(x_train_v, y_train)
report = classification_report(y_test, clf2.predict(x_test_v), target_names=categories)
print(report)


              precision    recall  f1-score   support

           1       0.78      0.12      0.21      3531
           2       0.20      0.59      0.30      1424
           3       0.21      0.34      0.26      1377
           4       0.29      0.45      0.35      1568
           5       0.89      0.58      0.71      4419

    accuracy                           0.41     12319
   macro avg       0.47      0.42      0.37     12319
weighted avg       0.63      0.41      0.42     12319



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Evaluation

**Evaluation metric:**
macro f1 score

Macro F1 score is a useful metric in scenarios where you want to evaluate the overall performance of a multi-class classification model, **particularly when the classes are imbalanced**

![Calculation](https://assets-global.website-files.com/5d7b77b063a9066d83e1209c/639c3d934e82c1195cdf3c60_macro-f1.webp)

In [ ]:
def calculate_macro_f1(y_true, y_pred, target_names=None):
  report_dict = classification_report(y_true, y_pred, target_names=target_names, output_dict=True)
  macro_f1 = report_dict['macro avg']['f1-score']
  return macro_f1

In [77]:
macro_f1 = calculate_macro_f1(y_test, clf2.predict(x_test_scaled), target_names=categories)
print("Macro F1-score:", macro_f1)


Macro F1-score: 0.4091625808804776


In [ ]:
macro_f1 = calculate_macro_f1(y_test, clf.predict(x_test_scaled), target_names=categories)
print("Macro F1-score:", macro_f1)


### Enhancement

In [64]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train_v)
x_train_scaled = scaler. transform(x_train_v)
x_test_scaled = scaler. transform(x_test_v)

In [67]:
clf3 = LogisticRegression(class_weight = class_weights, max_iter = 3000)
clf3.fit(x_train_scaled, y_train)
report = classification_report(y_test, clf3.predict(x_test_scaled), target_names=categories)
print(report)


              precision    recall  f1-score   support

           1       0.78      0.12      0.21      3531
           2       0.20      0.59      0.30      1424
           3       0.21      0.34      0.26      1377
           4       0.29      0.45      0.35      1568
           5       0.89      0.58      0.71      4419

    accuracy                           0.41     12319
   macro avg       0.47      0.42      0.37     12319
weighted avg       0.63      0.41      0.42     12319



In [76]:
macro_f1 = calculate_macro_f1(y_test, clf3.predict(x_test_scaled), target_names=categories)
print("Macro F1-score:", macro_f1)


Macro F1-score: 0.36654566759043217


### Conclusion and final results


#### Done!